In [ ]:
import pickle

import numpy as np
import open3d as o3d

import matplotlib.pyplot as plt
from matplotlib import cm

import ipywidgets as widgets
from IPython.display import display

In [ ]:
from tag_mapping.evaluation import LatticeNavigationGraph

from tag_mapping.utils import LineMesh, box_to_linemesh

In [ ]:
from helpers import generate_lattice_graph_shortest_path_linemeshes

In [ ]:
# for interactive dropdown selection
def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_option = change['new']

## Load evaluation output for a scene
Set `eval_output_path` to the path of the evaluation output file you want to inspect

In [ ]:
eval_output_path = 'eval_output.pkl'

with open(eval_output_path, 'rb') as f:
    eval_output = pickle.load(f)

Load and visualize the scene mesh

In [ ]:
scene_mesh = o3d.io.read_triangle_mesh(eval_output["ply_file_path"])

In [ ]:
o3d.visualization.draw_geometries([scene_mesh])

Load the scene lattice graph

In [ ]:
lattice_graph = LatticeNavigationGraph.load(eval_output["lattice_graph_path"])

Load other stored evaluation outputs

In [ ]:
label_gt_boxes = eval_output["label_gt_boxes"]
label_lattice_inds = eval_output["label_lattice_inds"]
label_gt_boxes_metrics = eval_output["label_gt_boxes_metrics"]
label_proposals = eval_output["label_proposals"]

Evaluation output stores box corners instead of the axis-aligned bounding box object as it's not pickleable, create the boxes again here

In [ ]:
for label, proposals in label_proposals.items():
    proposals["boxes"] = []
    for box_corners in proposals["boxes_corners"]:
        min_bound = np.min(box_corners, axis=0)
        max_bound = np.max(box_corners, axis=0)
        proposals["boxes"].append(
            o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
        )

## Visualize labeled ground-truth along with Tag Map localization
Use the dropdown menu to select a labeled semantic class to visualize

In [ ]:
select_mpcat40_label_dropdown = widgets.Dropdown(
    options=sorted(label_proposals.keys()),
    description='',
    disabled=False,
)
display(select_mpcat40_label_dropdown)

For the evaluation, a labeled semantic class is mapped to the set of corresponding tags in the vocabulary of the image tagging model. For example, the semantic class `chair` in Matterport3D is mapped to `office chair`, `beach chair`, and etc. The following dropdown can be used to visualize the localizations of a specific tag belonging to the selected semantic class.

If `all` is selected then the localizations of all tags corresponding to the selected semantic class are visualized.

The localization bounding boxes are colored by their confidence levels corresponding to the minimum number of votes for voxels within the bounding box.

In [ ]:
selected_label = select_mpcat40_label_dropdown.value
tag_options = list(set(label_proposals[selected_label]["tags"]))

select_tag_dropdown = widgets.Dropdown(
    options=['all'] + sorted(tag_options),
    description='',
    disabled=False,
)
display(select_tag_dropdown)

In [ ]:
label_boxes = label_gt_boxes[selected_label]
proposals = label_proposals[selected_label]

if len(proposals['boxes']) == 0:
    raise Exception("label has no associated proposals")

selected_tag = select_tag_dropdown.value
if selected_tag == 'all':
    selected_boxes = proposals["boxes"]
    selected_box_confidences = proposals["confidences"]
else:
    selected_boxes = [ box for i, box in enumerate(proposals["boxes"]) if proposals["tags"][i] == selected_tag]
    selected_box_confidences = [conf for i, conf in enumerate(proposals["confidences"]) if proposals["tags"][i] == selected_tag]

max_conf = np.max(selected_box_confidences)

viz_selected_boxes_linemeshes = []
for box, conf in zip(selected_boxes, selected_box_confidences):
    color = cm.viridis(conf/max_conf)[:3]
    viz_selected_boxes_linemeshes += box_to_linemesh(
        box, color=color, radius=0.02
    ).cylinder_segments
 
label_boxes_linemeshes = []
for box in label_boxes:
    label_boxes_linemeshes += box_to_linemesh(
        box, color=(0,1,0), radius=0.02
    ).cylinder_segments

o3d.visualization.draw_geometries([scene_mesh] + label_boxes_linemeshes + viz_selected_boxes_linemeshes)  

## Visualize the coarse-localization metrics

Use the following dropdown to select a semantic class to visualize

In [ ]:
select_mpcat40_label_dropdown = widgets.Dropdown(
    options=sorted(label_proposals.keys()),
    description='',
    disabled=False,
)
display(select_mpcat40_label_dropdown)

In [ ]:
selected_label = select_mpcat40_label_dropdown.value

### Visualize the P2E metric
Use the following dropdown to select a Tag Map proposed localization to visualize.

In [ ]:
proposals = label_proposals[selected_label]
proposals_confidences = proposals['confidences']
proposals_p2e = [m['p2e'] for m in proposals['metrics']]

select_proposal_options = []
for p2e, conf in zip(proposals_p2e, proposals_confidences):
    option = f"P2E: {p2e:.2f}   -   confidence: {conf}"
    select_proposal_options.append(option)

select_proposal_dropdown = widgets.Dropdown(
    options=select_proposal_options,
    description='',
    disabled=False,
)
display(select_proposal_dropdown)

In [ ]:
selected_proposal_index = select_proposal_dropdown.index

Visualize the selected Tag Map localization bounding box, the ground-truth bounding boxes for the selected semantic class, as well as the shortest paths from the localization to the ground-truth bounding boxes used in compute the P2E.

In [ ]:
if len(proposals["boxes"]) == 0:
    raise Exception("label has no associated Tag Map localizations")

proposal_box = proposals["boxes"][selected_proposal_index]
proposal_box_color = cm.viridis(
    proposals_confidences[selected_proposal_index] / np.max(proposals_confidences))[:3]
proposal_box_linemesh = box_to_linemesh(
    proposal_box, color=proposal_box_color, radius=0.02,
).cylinder_segments

proposal_node_inds = proposals["lattice_inds"][selected_proposal_index]
proposal_nodes_pcd = lattice_graph.o3d_nodes_pointcloud.select_by_index(proposal_node_inds)
proposal_nodes_pcd = proposal_nodes_pcd.paint_uniform_color((0,0,1))

label_boxes = label_gt_boxes[selected_label]
label_boxes_linemeshes = []
for box in label_boxes:
    label_boxes_linemeshes += box_to_linemesh(
        box, color=(0,1,0), radius=0.02
    ).cylinder_segments

label_node_inds = set()
for node_inds in label_lattice_inds[selected_label]:
        label_node_inds.update(node_inds)
label_node_inds = list(label_node_inds)
label_nodes_pcd = lattice_graph.o3d_nodes_pointcloud.select_by_index(label_node_inds)
label_nodes_pcd = label_nodes_pcd.paint_uniform_color((0,1,0))

shortest_path_linemeshes = generate_lattice_graph_shortest_path_linemeshes(
    lattice_graph, proposal_node_inds, label_node_inds
)

o3d.visualization.draw_geometries(
    [scene_mesh] + proposal_box_linemesh + [proposal_nodes_pcd] + label_boxes_linemeshes + [label_nodes_pcd] + shortest_path_linemeshes
)

### Visualize the E2P metric
Use the following dropdown to select a ground-truth bounding box of the selected semantic class to visualize.

In [ ]:
gt_boxes_e2p = [m['e2p'] for m in label_gt_boxes_metrics[selected_label]]

select_gt_box_options = []
for e2p in gt_boxes_e2p:
    option = f"E2P: {e2p:.2f}"
    select_gt_box_options.append(option)

select_gt_box_dropdown = widgets.Dropdown(
    options=select_gt_box_options,
    description='',
    disabled=False,
)
display(select_gt_box_dropdown)

In [ ]:
selected_gt_box_index = select_gt_box_dropdown.index

Visualize the selected ground-truth bounding box, the corresponding Tag Map localization for the selected semantic class, as well as the shortest paths from the selected ground-truth bounding box to the Tag Map localization bounding boxes.

In [ ]:
gt_box = label_gt_boxes[selected_label][selected_gt_box_index]
gt_box_linemesh = box_to_linemesh(
    gt_box, color=(0,1,0), radius=0.02,
).cylinder_segments

gt_box_node_inds = label_lattice_inds[selected_label][selected_gt_box_index]
gt_box_nodes_pcd = lattice_graph.o3d_nodes_pointcloud.select_by_index(gt_box_node_inds)
gt_box_nodes_pcd = gt_box_nodes_pcd.paint_uniform_color((0,1,0))

proposal_boxes = label_proposals[selected_label]["boxes"]
if len(proposal_boxes) == 0:
    raise Exception("label has no associated Tag Map localizations")

proposal_boxes_linemeshes = []
for i, box in enumerate(proposal_boxes):
    color = cm.viridis(
        label_proposals[selected_label]['confidences'][i] / np.max(label_proposals[selected_label]['confidences'])
    )[:3]
    proposal_boxes_linemeshes += box_to_linemesh(
        box, color=color, radius=0.02
    ).cylinder_segments

proposal_boxes_node_inds = set()
for node_inds in label_proposals[selected_label]["lattice_inds"]:
        proposal_boxes_node_inds.update(node_inds)
proposal_boxes_node_inds = list(proposal_boxes_node_inds)
proposal_boxes_nodes_pcd = lattice_graph.o3d_nodes_pointcloud.select_by_index(proposal_boxes_node_inds)
proposal_boxes_nodes_pcd = proposal_boxes_nodes_pcd.paint_uniform_color((0,0,1))

shortest_path_linemeshes = generate_lattice_graph_shortest_path_linemeshes(
    lattice_graph, gt_box_node_inds, proposal_boxes_node_inds
)

o3d.visualization.draw_geometries(
    [scene_mesh] + gt_box_linemesh + [gt_box_nodes_pcd] + proposal_boxes_linemeshes + [proposal_boxes_nodes_pcd] + shortest_path_linemeshes
)